# Building a database for crime reports

In this guided project, we will build a database for storing data related with crimes that occurred in Boston. This dataset is available in the file boston.csv.

In [1]:
import csv
import pandas as pd
data=pd.read_csv('boston.csv')
data.head()

,incident_number,offense_code,description,date,day_of_the_week,lat,long
0,1,619,LARCENY ALL OTHERS,2018-09-02,Sunday,42.357791,-71.139371
1,2,1402,VANDALISM,2018-08-21,Tuesday,42.306821,-71.060300
2,3,3410,TOWED MOTOR VEHICLE,2018-09-03,Monday,42.346589,-71.072429
3,4,3114,INVESTIGATE PROPERTY,2018-09-03,Monday,42.334182,-71.078664
4,5,3114,INVESTIGATE PROPERTY,2018-09-03,Monday,42.275365,-71.090361


The first column represents the identifier of the crime. The second contains a numeric identifier code for the committed crime. The third represents a description of the crime. The next two rows contain the date on which the crime happened and the corresponding day of the week. Finally, the last two columns represent the location of the crime with a latitude and longitude coordinates.

The goal of this project is to create a database named crimes_db with a table – boston_crimes – with appropriate datatypes for storing the data from the boston.csv file.

We will be creating the table inside a schema named crimes. We will also create the readonly and readwrite groups with the appropriate privileges. Finally, we will also need to create one user for each of these groups.

## Create the database and the schema

We will start by creating a database for storing our crime data as well as a schema for containing the tables.

In [2]:
import psycopg2
conn = psycopg2.connect("dbname = dq user = dq")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.close()

In [3]:
conn = psycopg2.connect("dbname = crime_db user = dq")
conn.autocommit = True
cur=conn.cursor()
cur.execute("CREATE SCHEMA crimes;")

## Obtaining the Column Names and Sample

We now have a database and a schema — we are ready to start creating tables. Before we do that, let's gather some data about our crime dataset so that we can more easily select the right datatypes to use in our table.

Let's start by reading the column names from the boston.csv file as well as the first row. In this way we will have them at hand throughout this guided project so that we can easily take a look at them at any moment.

In [4]:
with open("boston.csv") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating a function for analyzing column values

We now have access to the column headers and the first row of data. This will help us when we create the table as we will not have to remember the names of the columns and using the first row of data we can easily recall what kind of data is stored in each column.

Before we create a table for storing the crime data, we need to identify the proper datatypes for the columns. To help us with that, let's create a function — get_col_value_set() — that given the name of a CSV file and a column index (starting a 0) that computes a Python set with all distinct values contained in that column.

This function will be useful for two reasons:

- Checking whether an enumerated datatype might be a good choice for representing a column.
- Computing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns.

In [5]:
def get_col_value_set(csv_filename, col_index):
    import csv
    values = set()
    with open(csv_filename, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_value_set("boston.csv", i)
    print(col_headers[i], len(values), sep = '\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


## Analyzing the maximum length of the description column

With the function from the previous screen we can compute the number of distinct values for each column. Columns with a low number of distinct values tend to be good candidates for enumerated datatypes.

There are two textual column in the data set, namely, the description and day_of_the_week columns. However the day of the week contains only 7 values, one for each day. We can tell that the longest of them is Wednesday without needing any computation.

In [6]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


Description is at index 2.

In [7]:
descriptions = get_col_value_set("boston.csv", 2)
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))

print(max_len)

58


## Creating the table

Let's create a table for storing the Boston crime data. Remember the information that we have gathered in the two previous screens, namely, the number of different values of each column and the maximum length of the string columns.

We will create an enumerated datatype named weekday for the day_of_the_week since there there only seven possible values.

For the incident_number we have decided to user the type INTEGER and set it as the primary key. The same datatype was also used to represent the offense_code.

Since the description has at most 58 character we decided to use the datatype VARCHAR(100) for representing it. This leave some margin while not being so big that we will waste a lot of memory.

The date was represented as the DATE datatype. Finally, for the latitude and longitude we used DECIMAL datatypes.


We will use the same names for the column headers.

In [8]:
cur.execute("CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');")

In [9]:
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
    incident_number INTEGER PRIMARY KEY,
    offense_code INTEGER,
    description VARCHAR(100),
    date DATE,
    day_of_the_week weekday,
    lat decimal,
    long decimal);""")

## Load the data into the table

We used the copy_expert to load the data as it is very fast and very succinct to use.

In [10]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
cur.execute("SELECT * FROM crimes.boston_crimes")
print(len(cur.fetchall()))

298329


## Revoke public privileges

Our database is starting to look good! We have created a database with a schema inside it for hold data about crimes. We selected the right datatypes for storing the data, created a table and loaded the CSV containing crimes about Boston.

Now it is time to handle users. Our goal is to create the two user groups: readonly and readwrite. By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the public group and on the public schema.

In [11]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Creating the read only and read write  groups

We have made sure that we are not going to inadvertently inherit privileges from the public group. The next step is to create our two users groups.

That the readonly group is supposed to only privileges to perform SELECT queries. In contrast, we want the readwrite group to be able to perform SELECT, INSERT, DELETE and UPDATE queries. To grant INSERT and SELECT privileges in all tables inside a schema we can use the command:

Both these uses will also need the following:

- Connection privileges on the crime_db — otherwise, they won't be able to do anything.
- Usage of the crimes schema — this is where all data will be stored.

In [13]:
cur.execute("CREATE GROUP readonly WITH NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

In [14]:
cur.execute("CREATE GROUP readwrite WITH NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

## Creating one user for each group

Let's wrap up the project by creating one user in each group. We will need to create each user and then assign them to each group. 

In [20]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")

cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

## Test the database setup

Test the database setup using SQL queries on the pg_roles table and information_schema.table_privileges.

In the pg_roles table we will check database related privileges and for that we will look at the following columns:

- rolname: The name of the user / group that the privilege refers to.
- rolsuper: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
- rolcreaterole: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
- rolcreatedb: Whether user / group can create databases. It should be False on every user / group that we have created.
- rolcanlogin: Whether user / group can login. It should be True on the users and False on the groups that we have created.

In the information_schema.table_privileges we will check privileges related to SQL queries on tables. We will list the privileges of each group that we have created.

In [ ]:
conn.close()

conn = psycopg2.connect("dbname = crim_db user = dq")

cur = conn.cursor()
cur.execute("""
SELECT rolname, rolsuper, rolcreaterol, rolcreatedb, rolcanlogin
FROM pg_roles
WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');""")

cur.execute("""
SELECT grantee, privilege_type
FROM information_schema.table_privileges
WHERE grantee = 'readwrite'""")